In [1]:
import re
from copy import deepcopy
import xml.etree.ElementTree as ET

In [7]:
def species_txt_to_dict(txt_file):
    #read the file
    file = open(txt_file,'r')
    lines = file.readlines()
    file.close()
    species_info = {}

    #iterate over lines
    for line in lines:
        #spilt the line
        strings = line.split()

        #skip if string is empty array
        if len(strings) <1:
            continue  

        #find line with species
        if strings[-1] == '1':
            #get species name
            specie = strings[0]
            specie_state = strings[1]

            #get the low temp min and max
            low_min = strings[-4]
            low_max = 1000.000

            #get the high temp min and max
            high_min = 1000.000
            high_max = strings[-3] 

        if strings[-1] == '2': 
            #spilt the line by number as the numbers are not broken up by spaces in txt file
            strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)

            #get first 5 high coefs
            high_coeffs = []
            high_coeffs.extend(strings[0:-1])


        if strings[-1] == '3': 
            #spilt the line by number as the numbers are not broken up by spaces in txt file
            strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)

            #first two are high coeefs
            high_coeffs.extend(strings[0:2])

            #remaining are low coeffs
            low_coeffs = []
            low_coeffs.extend(strings[2:-1])

        if strings[-1] == '4': 
            #spilt the line by number as the numbers are not broken up by spaces in txt file
            strings = re.findall(r"[+\-]?(?:0|[1-9]\d*)(?:\.\d*)?(?:[eE][+\-]?\d+)?", line)

            # get low coefs
            low_coeffs.extend(strings[0:-1])

            #Add to dictionary
            species_info[specie,specie_state]={'low':{},'high':{}}
            species_info[specie,specie_state]['low']['Tmax'] = low_max
            species_info[specie,specie_state]['low']['Tmin'] = low_min
            species_info[specie,specie_state]['low']['coeffs'] = low_coeffs
            species_info[specie,specie_state]['high']['Tmax'] = high_max
            species_info[specie,specie_state]['high']['Tmin'] = high_min
            species_info[specie,specie_state]['high']['coeffs'] = high_coeffs
    return species_info

In [40]:
from xml.etree import ElementTree
from xml.dom import minidom

def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

from xml.etree.ElementTree import Element, SubElement, Comment
from xml.etree.ElementTree import XMLID

species_info = species_txt_to_dict('7poly_scrapper_output.txt')

root = Element('specieData')
root.set('id','specie_data')
comment = Comment('Created by Riddhi Shah')
root.append(comment)

for i,k in enumerate(species_info):

    specie = Element('specie')
    specie.set('name',str(k[0]))
    specie.set('state',str(k[1]))
    specie.set('Mweight','N/A')
    
    thermo = SubElement(specie, 'thermo')

    NASA_low = SubElement(thermo, 'NASA')
    NASA_low.set('Tmin',str(species_info[k]['low']['Tmin']))
    NASA_low.set('Tmax',str(species_info[k]['low']['Tmax']))
    floatArray = SubElement(NASA_low, 'floatArray')
    floatArray.set('name','coeffs')
    low_coeffs =", ".join(species_info[k]['low']['coeffs'])
    floatArray.text = low_coeffs
             
    NASA_HIGH = SubElement(thermo, 'NASA')
    NASA_HIGH.set('Tmin',str(species_info[k]['high']['Tmin']))
    NASA_HIGH.set('Tmax',str(species_info[k]['high']['Tmax']))
    floatArray = SubElement(NASA_HIGH, 'floatArray')
    floatArray.set('name','coeffs')
    high_coeffs =", ".join(species_info[k]['high']['coeffs'])
    floatArray.text = high_coeffs
    
    root.append(specie)

result = prettify(root)
f = open("7poly.xml","w")
f.write(result)
f.close()


print(result)

<?xml version="1.0" ?>
<specieData id="specie_data">
  <!--Created by Riddhi Shah-->
  <specie Mweight="N/A" name="Ag" state="(solid)">
    <thermo>
      <NASA Tmax="1000.0" Tmin="1235.080">
        <floatArray name="coeffs">2.25225065E+00, 5.43263008E-03, -1.32153990E-05, 1.50423505E-08, -5.94991675E-12, -8.23132027E+02, -8.86835190E+00, 0.00000000E+00</floatArray>
      </NASA>
      <NASA Tmax="A" Tmin="1000.0">
        <floatArray name="coeffs">2.07216824E+00, 2.46393729E-03, -1.34351116E-06, 3.69321107E-10, 0.00000000E+00, -6.37725170E+02, -7.18810718E+00</floatArray>
      </NASA>
    </thermo>
  </specie>
  <specie Mweight="N/A" name="Ag" state="(liquid)">
    <thermo>
      <NASA Tmax="1000.0" Tmin="6000.000">
        <floatArray name="coeffs">0.00000000E+00, 0.00000000E+00, 0.00000000E+00, 0.00000000E+00, 0.00000000E+00, 0.00000000E+00, 0.00000000E+00, 0.00000000E+00</floatArray>
      </NASA>
      <NASA Tmax="A" Tmin="1000.0">
        <floatArray name="coeffs">4.04091552E+0

In [46]:
#Import SQLlite
import sqlite3
import numpy as np
import pandas as pd

def create_tables():

    pd.set_option('display.width', 500)
    pd.set_option('display.max_columns', 100)
    pd.set_option('display.notebook_repr_html', True)

    db = sqlite3.connect('HW10_demo.sqlite')
    cursor = db.cursor()
    cursor.execute("DROP TABLE IF EXISTS LOW")
    cursor.execute("DROP TABLE IF EXISTS HIGH")
    cursor.execute("PRAGMA foreign_keys=1")

    #Create High and Low tables
    cursor.execute('''CREATE TABLE LOW ( 
                   SPECIES_NAME TEXT NOT NULL, 
                   TLOW TEXT NOT NULL, 
                   THIGH TEXT NOT NULL, 
                   COEFF_1 TEXT NOT NULL,
                   COEFF_2 TEXT NOT NULL,
                   COEFF_3 TEXT NOT NULL,
                   COEFF_4 TEXT NOT NULL,
                   COEFF_5 TEXT NOT NULL,
                   COEFF_6 TEXT NOT NULL,
                   COEFF_7 TEXT NOT NULL,
                   COEFF_8 TEXT NOT NULL)''')

    # Commit changes to the database
    db.commit()
    cursor.execute('''CREATE TABLE HIGH (
                   SPECIES_NAME TEXT NOT NULL, 
                   TLOW TEXT NOT NULL, 
                   THIGH TEXT NOT NULL, 
                   COEFF_1 TEXT NOT NULL,
                   COEFF_2 TEXT NOT NULL,
                   COEFF_3 TEXT NOT NULL,
                   COEFF_4 TEXT NOT NULL,
                   COEFF_5 TEXT NOT NULL,
                   COEFF_6 TEXT NOT NULL,
                   COEFF_7 TEXT NOT NULL)''')
    db.commit()

In [61]:
#Parse XML to get info for each species
import xml.etree.ElementTree as ET

def species_xml_to_db():
    
    #Get the xml
    tree = ET.parse('7poly.xml')
    root = tree.getroot()

    #get species
    species = root.findall('specie')

    for specie in species:
        name = specie.get('name')

        #get low temp high/low and coeffs for each specie
        NASA = specie.find('thermo').findall('NASA')

        #get low info
        low_tmax = NASA[0].get('Tmax')
        low_tmin = NASA[0].get('Tmin')

        #to handle where there are 8 low coeffs
        lows = NASA[0].find('floatArray').text.split()
        if(len(lows) > 7):
            Low_C_1,Low_C_2,Low_C_3,Low_C_4,Low_C_5,Low_C_6,Low_C_7,Low_C_8 = lows[0:8]
            lows_to_insert = (name,low_tmin,low_tmax,Low_C_1.strip(','),Low_C_2.strip(','),Low_C_3.strip(','),Low_C_4.strip(','),Low_C_5.strip(','),Low_C_6.strip(','),Low_C_7.strip(','),Low_C_8.strip(','))
        else:
            Low_C_1,Low_C_2,Low_C_3,Low_C_4,Low_C_5,Low_C_6,Low_C_7 = lows[0:7]
            lows_to_insert = (name,low_tmin,low_tmax,Low_C_1.strip(','),Low_C_2.strip(','),Low_C_3.strip(','),Low_C_4.strip(','),Low_C_5.strip(','),Low_C_6.strip(','),Low_C_7.strip(','),"")

        #get low info
        high_tmax = NASA[1].get('Tmax')
        high_tmin = NASA[1].get('Tmin')
        High_C_1,High_C_2,High_C_3,High_C_4,High_C_5,High_C_6,High_C_7 = NASA[1].find('floatArray').text.strip(',').split()[0:7]
        high_to_insert = name,high_tmin,high_tmax,High_C_1.strip(','),High_C_2.strip(','),High_C_3.strip(','),High_C_4.strip(','),High_C_5.strip(','),High_C_6.strip(','),High_C_7.strip(',')

        #Insert the values for each species into table
        cursor.execute('''INSERT INTO LOW 
                      (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7, COEFF_8)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', lows_to_insert)
        cursor.execute('''INSERT INTO HIGH 
                      (SPECIES_NAME, TLOW, THIGH, COEFF_1, COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''', high_to_insert)

<Element 'specieData' at 0x1034f5a98>


In [62]:
def viz_tables(cols, query):
    q = cursor.execute(query).fetchall()
    framelist = []
    for i, col_name in enumerate(cols):
        framelist.append((col_name, [col[i] for col in q]))
    return pd.DataFrame.from_items(framelist)

In [63]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(LOW)")]
query = '''SELECT * FROM LOW'''
viz_tables(candidate_cols, query)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7,COEFF_8
0,Ag,1235.080,1000.0,2.25225065E+00,5.43263008E-03,-1.32153990E-05,1.50423505E-08,-5.94991675E-12,-8.23132027E+02,-8.86835190E+00,0.00000000E+00
1,Ag,6000.000,1000.0,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00
2,Ag,6000.000,1000.0,2.50000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,3.35200237E+04,6.56281935E+00,3.42653987E+04
3,Ag+,6000.000,1000.0,1.67072904E-01,2.30034783E-02,-8.05675031E-05,1.14647031E-07,-5.08119362E-11,1.22365909E+05,1.49717871E+01,1.22928919E+05
4,Ag-,6000.000,1000.0,2.50067694E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,1.76654697E+04,5.86586664E+00,1.84110465E+04
5,AIR,6000.000,1000.0,3.56839620E+00,-6.78729429E-04,1.55371476E-06,-3.29937060E-12,-4.66395387E-13,-1.06234659E+03,3.71582965E+00,-1.50965000E+01
6,AL(cr),933.610,1000.0,1.01032822E+00,1.20775640E-02,-2.62098556E-05,2.64295083E-08,-9.01946533E-12,-6.54447545E+02,-5.00441242E+00,0.00000000E+00
7,AL(L),6000.000,1000.0,3.82018990E+00,-1.67563330E-06,0.00000000E+00,0.00000000E+00,0.00000000E+00,-9.57094068E+01,-1.75321254E+01,0.00000000E+00
8,AL,6000.000,1000.0,3.11146855E+00,-3.57363458E-03,8.02692389E-06,-7.89637203E-09,2.83934686E-12,3.88641504E+04,2.83632371E+00,3.96896510E+04
9,AlB2,1300.000,1000.0,-3.37206697E+00,4.86717911E-02,-8.64112411E-05,7.48292677E-08,-2.43156002E-11,-1.86914266E+04,1.21055076E+01,-1.81605410E+04


In [60]:
candidate_cols = [col[1] for col in cursor.execute("PRAGMA table_info(HIGH)")]
query = '''SELECT * FROM HIGH'''
viz_tables(candidate_cols, query)

,SPECIES_NAME,TLOW,THIGH,COEFF_1,COEFF_2,COEFF_3,COEFF_4,COEFF_5,COEFF_6,COEFF_7
0,Ag,1000.0,A,2.07216824E+00,2.46393729E-03,-1.34351116E-06,3.69321107E-10,0.00000000E+00,-6.37725170E+02,-7.18810718E+00
1,Ag,1000.0,A,4.04091552E+00,-3.49297186E-05,1.60169701E-08,-2.96225835E-12,1.92332513E-16,-4.77718035E+02,-1.78491707E+01
2,Ag,1000.0,A,2.49722745E+00,-5.96378605E-07,7.16732589E-09,-4.57774728E-12,7.79345485E-16,3.35216939E+04,6.58031521E+00
3,Ag+,1000.0,B,9.72687035E+00,-4.01472180E-03,7.47796464E-07,-1.76595533E-11,-4.14279861E-15,1.17714726E+05,-3.91847888E+01
4,Ag-,1000.0,C,2.50067694E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,1.76654697E+04,5.86586664E+00
5,AIR,1000.0,B,3.08792717E+00,1.24597184E-03,-4.23718945E-07,6.74774789E-11,-3.97076972E-15,-9.95262755E+02,5.95960930E+00
6,AL(cr),1000.0,B,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00
7,AL(L),1000.0,B,3.83089866E+00,-2.09027129E-05,1.04271684E-08,-2.04841051E-12,1.39565517E-16,-9.97961566E+01,-1.75914374E+01
8,AL,1000.0,B,2.53152311E+00,-4.16002811E-05,2.40955537E-08,-7.15420953E-12,8.68511135E-16,3.89410793E+04,5.39170504E+00
9,AlB2,1000.0,C,3.39901617E+00,9.31632398E-03,-4.49474072E-06,1.18155112E-09,0.00000000E+00,-1.95414672E+04,-1.77999219E+01
